In [37]:
using CSV, DataFrames
# D = demand matrix (day of the week)
# n locations
# W = foot-traffic matrix (for each location, foot traffic for day of the week)

D = CSV.read("electricity/demand.csv", DataFrame; header=true)[:, 2]#7x1
mean_counts_df = CSV.read("ped_traffic/mean_counts.csv", DataFrame; header=true)
W = mean_counts_df[:, 3:9]|> Matrix #110x7
location_ids = mean_counts_df[:, 1] # location IDs

mean_counts_df2 = CSV.read("ped_traffic/mean_counts_dist.csv", DataFrame; header=true)
W2 = mean_counts_df2[:, 3:9]|> Matrix #44x7
d = mean_counts_df2[:, 10] #44x1
location_ids2 = mean_counts_df2[:, 1] # location IDs

println(size(D))
println(size(W))
println(size(W2))
println(size(d))

(7,)
(110, 7)
(44, 7)
(44,)


In [38]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations(D::Vector{Float64}, W::Matrix{Float64})

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    @objective(model, Min, sum(x[i] for i in 1:n)) # objective func

    for t in 1:T # constraint each day
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    min_num_locations = length(chosen_locations)

    return min_num_locations, chosen_locations, model
end

optimize_installations (generic function with 1 method)

In [39]:
min_locs, chosen, model = optimize_installations(D, W)
println("Minimum number of locations needed: ", min_locs)
println("Chosen location indices: ", chosen)

Minimum number of locations needed: 4
Chosen location indices: [15, 17, 19, 78]


In [ ]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations_dist(D::Vector{Float64}, W::Matrix{Float64}, d::Vector{Float64})

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"
    @assert length(d) == n "Distance vector d must have length n"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    # Objective: minimize e^distance - 1 for selected locations
    @objective(model, Min, sum((exp(d[i] * x[i]) - 1) for i in 1:n))

    # Constraint: maximum 5 locations can be selected
    @constraint(model, sum(x[i] for i in 1:n) ≤ 5)

    # Constraint: energy production must meet demand each day
    for t in 1:T
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    total_distance = sum(d[i] for i in chosen_locations)
    num_locations = length(chosen_locations)

    return num_locations, chosen_locations, total_distance, model
end

optimize_installations_dist (generic function with 1 method)

In [41]:
# Convert distance vector from Matrix to Vector
d_vec = vec(d)

# Call optimize_installations_dist with distance vector
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, d_vec)

# Calculate total objective value (e^distance - 1) for selected locations
total_objective = sum(exp(d_vec[i]) - 1 for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value (e^distance - 1): ", total_objective)

LoadError: Invalid coefficient Inf on variable x[23].

In [33]:
# Calculate energy produced vs needed for each day
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
energy_produced = zeros(7)
energy_needed = percent_demand .* D

println(chosen)

for t in 1:7
    for i in chosen
        # energy_produced[t] += W[i, t] * 0.1
        energy_produced[t] += W2[i, t] * 0.1
    end
end

# Create comparison DataFrame
energy_comparison = DataFrame(
    Day = days,
    Energy_Produced = energy_produced,
    Energy_Needed = energy_needed,
    Surplus = energy_produced .- energy_needed,
    # Coverage_Percent = (energy_produced ./ energy_needed) .* 100
)

println("=" ^ 70)
println("Energy Production vs Demand Analysis")
println("=" ^ 70)
println(energy_comparison)
println()


[6, 9, 10, 34]
Energy Production vs Demand Analysis
7×4 DataFrame
 Row │ Day        Energy_Produced  Energy_Needed  Surplus   
     │ String     Float64          Float64        Float64   
─────┼──────────────────────────────────────────────────────
   1 │ Monday             1197.35        1144.2    53.1456
   2 │ Tuesday            1197.35        1181.74   15.6051
   3 │ Wednesday          1197.35        1191.14    6.20684
   4 │ Thursday           1197.35        1190.01    7.33593
   5 │ Friday             1197.35        1161.43   35.9159
   6 │ Saturday           1197.35        1054.46  142.891
   7 │ Sunday             1197.35        1061.85  135.502



In [10]:
# Calculate total foot traffic per location (sum across all days)
total_foot_traffic = [sum(W[i, :]) for i in 1:size(W, 1)]

# Create DataFrame with indices and total foot traffic
location_traffic = DataFrame(
    Index = 1:size(W, 1),
    Total_Foot_Traffic = total_foot_traffic
)

# Sort by total foot traffic (descending)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

Top 10 Locations by Total Foot Traffic
10×2 DataFrame
 Row │ Index  Total_Foot_Traffic 
     │ Int64  Float64            
─────┼───────────────────────────
   1 │    15            34608.0
   2 │    20            24220.0
   3 │    19            20345.5
   4 │     9            18670.8
   5 │    78            16989.0
   6 │    75            14568.8
   7 │    17            14199.5
   8 │    31            14115.6
   9 │    57            11077.5
  10 │    58             9805.69


In [34]:
# Calculate total foot traffic per location (sum across all days)
total_foot_traffic = [sum(W2[i, :]) for i in 1:size(W2, 1)]

# Create DataFrame with indices and total foot traffic
location_traffic = DataFrame(
    Index = 1:size(W2, 1),
    Total_Foot_Traffic = total_foot_traffic
)

# Sort by total foot traffic (descending)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

Top 10 Locations by Total Foot Traffic
10×2 DataFrame
 Row │ Index  Total_Foot_Traffic 
     │ Int64  Float64            
─────┼───────────────────────────
   1 │     6            34608.0
   2 │    10            24220.0
   3 │     9            20345.5
   4 │    24            14568.8
   5 │     8            14199.5
   6 │    20            11077.5
   7 │    33             9224.25
   8 │     7             5005.0
   9 │    36             4739.0
  10 │    34             4641.0
